In [1]:
import os
os.chdir("..")
os.getcwd()

'/home/l727n/Projects/ml_perovskite'

In [2]:
import torch
import torch.nn as nn
import numpy as np
import kaleido
from torch.utils.data import DataLoader
from data.perovskite_dataset import (
    PerovskiteDataset1d,
    PerovskiteDataset2d,
    PerovskiteDataset3d,
    PerovskiteDataset2d_time,
)
from models.resnet import ResNet152, ResNet, BasicBlock, Bottleneck
from models.slowfast import SlowFast
from data.augmentations.perov_1d import normalize
from data.augmentations.perov_2d import normalize as normalize_2d
from data.augmentations.perov_3d import normalize as normalize_3d
from base_model import seed_worker
from argparse import ArgumentParser
from os.path import join

data_dir = "/home/l727n/Projects/Applied Projects/ml_perovskite/preprocessed"
checkpoint_dir = "/home/l727n/E132-Projekte/Projects/Helmholtz_Imaging_ACVL/KIT-FZJ_2021_Perovskite/data_Jan_2022/checkpoints"

path_to_checkpoint = join(
    checkpoint_dir, "1D_no_border-epoch=999-val_MAE=0.000-train_MAE=0.351.ckpt"
)

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import of model and computation of six different attribution methods with two evaluation metrics per method

In [3]:
#### 1D Model (no border) 

hypparams = {
    "dataset": "Perov_1d",
    "dims": 1,
    "bottleneck": False,
    "name": "ResNet18",
    "data_dir": data_dir,
    "no_border": True,
    "resnet_dropout": 0.0,
}

model = ResNet.load_from_checkpoint(
    path_to_checkpoint,
    block=BasicBlock,
    num_blocks=[2, 2, 2, 2],
    num_classes=1,
    hypparams=hypparams,
)

print("Loaded")
model.eval()

dataset = PerovskiteDataset1d(
    data_dir,
    transform=normalize(model.train_mean, model.train_std),
    scaler=model.scaler,
    no_border=True,
)

batch_size = 256

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    worker_init_fn=seed_worker,
    persistent_workers=True,
)

Load data
tensor([0.2951, 0.0195, 0.0058, 0.0218]) tensor([0.1621, 0.0112, 0.0030, 0.0151])
Loaded


In [4]:
# Select observation
n = 1

x_batch = next(iter(loader))
x = x_batch[0][n]

with torch.no_grad():
    y_batch = model.predict(x_batch).flatten()

y = float(np.round(y_batch[n].detach().numpy(),2))

In [5]:
# Init pertubation function for infidelity metric 

std_noise = 0.01

def perturb_fn(inputs):
    noise = torch.tensor(np.random.normal(0, std_noise, inputs.shape)).float()
    return noise, inputs - noise

In [20]:
# Compute Attribution via expected gradients

from captum.attr import GradientShap
from captum.metrics import sensitivity_max, infidelity

gradient_shap = GradientShap(model)
attr_eg = gradient_shap.attribute(            
    x_batch[0][n].unsqueeze(0),
    n_samples=100,
    stdevs=0.001,
    baselines= x_batch[0],
    target=0,
)

infid_eg = infidelity(model, perturb_fn, x_batch[0][n].unsqueeze(0), attr_eg)
sens_eg = sensitivity_max(gradient_shap.attribute, x_batch[0][n].unsqueeze(0), target = 0, baselines= x_batch[0]) # lower is better

In [25]:
# Integrated Gradients

from captum.attr import IntegratedGradients

ig = IntegratedGradients(model)
attr_ig, delta = ig.attribute(x_batch[0][n].unsqueeze(0), baselines=x_batch[0][n].unsqueeze(0) * 0, return_convergence_delta=True)

infid_ig = infidelity(model, perturb_fn, x_batch[0][n].unsqueeze(0), attr_ig)
sens_ig = sensitivity_max(ig.attribute, x_batch[0][n].unsqueeze(0), target = 0, baselines= x_batch[0][n].unsqueeze(0) * 0)

In [26]:
# LIME

from captum.attr import Lime
from torch import Tensor
from captum._utils.models.linear_model import SkLearnLinearModel

a = 0.01
kernel_width=7.0

def similarity_kernel(
    original_input,
    perturbed_input,
    perturbed_interpretable_input,
    **kwargs)->Tensor:

        if type(perturbed_input) == tuple:
            original_input= original_input[0]
            perturbed_input = perturbed_input[0]

        # kernel_width will be provided to attribute as a kwarg
        l2_dist = torch.norm(original_input - perturbed_input)
        return torch.exp(- (l2_dist**2) / (kernel_width**2))

lime = Lime(model,SkLearnLinearModel("linear_model.Lasso", alpha=a), similarity_func = similarity_kernel)
attr_lime = lime.attribute(inputs = x_batch[0][n].unsqueeze(0), n_samples=500,perturbations_per_eval = 20)

infid_lime = infidelity(model, perturb_fn, x_batch[0][n].unsqueeze(0), attr_lime)
sens_lime = sensitivity_max(lime.attribute, x_batch[0][n].unsqueeze(0), n_samples=500,perturbations_per_eval = 20)

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/captum/attr/_core/lime.py:1110: UserWarning:

You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.



In [29]:
# Kernel SHAP

from captum.attr import KernelShap, NoiseTunnel

stdev = 8.0

ks = KernelShap(model)
nt = NoiseTunnel(ks)
attr_ks = nt.attribute(x_batch[0][n].unsqueeze(0),target = 0, n_samples=100, nt_samples=10,stdevs = stdev, nt_type='smoothgrad')

infid_ks = infidelity(model, perturb_fn, x_batch[0][n].unsqueeze(0), attr_ks)
sens_ks = sensitivity_max(nt.attribute, x_batch[0][n].unsqueeze(0), n_samples=100,nt_samples=10,stdevs = stdev, nt_type='smoothgrad')

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/captum/attr/_core/lime.py:1110: UserWarning:

You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/captum/attr/_core/lime.py:1110: UserWarning:

You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/captum/attr/_core/lime.py:1110: UserWarning:

You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributi

In [21]:
# Guided Backprob

from captum.attr import GuidedBackprop

gbp = GuidedBackprop(model)
attr_gbp = gbp.attribute(x_batch[0][n].unsqueeze(0),target = 0)

infid_gbp = infidelity(model, perturb_fn, x_batch[0][n].unsqueeze(0), attr_gbp)
sens_gbp = sensitivity_max(gbp.attribute, x_batch[0][n].unsqueeze(0))

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/captum/_utils/gradient.py:57: UserWarning:

Input Tensor 0 did not already require gradients, required_grads has been set automatically.

/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning:

Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished



In [22]:
# Guided Backprob

from captum.attr import GuidedGradCam

ggc = GuidedGradCam(model, model.conv1)
attr_ggc = ggc.attribute(x_batch[0][n].unsqueeze(0),target = 0)
attr_ggc = attr_ggc.detach()

infid_ggc = infidelity(model, perturb_fn, x_batch[0][n].unsqueeze(0), attr_ggc)
sens_ggc = sensitivity_max(ggc.attribute, x_batch[0][n].unsqueeze(0))

# Visualization of single methods and comparision of methods via sum and absolut values

In [19]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=4, 
                    specs=[[{"secondary_y": True, "colspan": 4}, None,None,None],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Total Attribution"), format_title("", "Attribution ND"), format_title("", "Attribution LP725"), format_title("", "Attribution LP780"), format_title("", "Attribution SP775"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND", marker_color="#042940"))
fig.add_trace(go.Scatter(y=x[1], name = "LP725", marker_color="#005C53"))
fig.add_trace(go.Scatter(y=x[2], name = "LP780", marker_color="#9FC131"))
fig.add_trace(go.Scatter(y=x[3], name = "SP775", marker_color="#DBF227"))

fig.add_trace(go.Bar(y=attr_eg.squeeze().sum(axis=0), name = "Attribution",marker_color=np.where(attr_eg.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False),secondary_y=True)

fig.update_yaxes(title= "Intensity")
fig.update_yaxes(title_text="Attribution", secondary_y=True)
fig.update_xaxes(title= "Timestep")

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y=attr_eg.squeeze()[0],marker_color=np.where(attr_eg.squeeze()[0]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.update_yaxes(title= None, row=2, col=1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=1)
fig.update_xaxes(title= None, row=2, col=1)

fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_eg.squeeze()[1],marker_color=np.where(attr_eg.squeeze()[1]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.update_yaxes(title= None, row=2, col=2)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=2)
fig.update_xaxes(title= None, row=2, col=2)

fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_eg.squeeze()[2],marker_color=np.where(attr_eg.squeeze()[2]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None, row=2, col=3)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=3)
fig.update_xaxes(title= None, row=2, col=3)

fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=4)
fig.add_trace(go.Bar(y=attr_eg.squeeze()[3],marker_color=np.where(attr_eg.squeeze()[3]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=4)

fig.update_yaxes(title= None, row=2, col=4)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=4)
fig.update_xaxes(title= None, row=2, col=4)

fig.update_layout(
    title= format_title("Perovskite 1D Model (Backflow patches excluded)", 
    "Predicted PCE: " + str(y) + " / Method: Expected Gradients / Infidelity = " + str(*np.round(infid_eg.numpy(),4)) + " ("+ u"\u03C3" + "(" + u"\u03B5" + ") = " + str(std_noise) + ")" + " / Sensitivity = " + str(*np.round(sens_eg.numpy(),4))),
    legend_title = "Wavelength",
    title_y = 0.965,
    title_x = 0.035,
    template = "plotly_white",
    height=600, width=2200
)

fig.write_image("xai/images/1D/1D_nb_eg.png", scale=2)

fig.show()


In [27]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=4, 
                    specs=[[{"secondary_y": True, "colspan": 4}, None,None,None],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Total Attribution"), format_title("", "Attribution ND"), format_title("", "Attribution LP725"), format_title("", "Attribution LP780"), format_title("", "Attribution SP775"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND", marker_color="#042940"))
fig.add_trace(go.Scatter(y=x[1], name = "LP725", marker_color="#005C53"))
fig.add_trace(go.Scatter(y=x[2], name = "LP780", marker_color="#9FC131"))
fig.add_trace(go.Scatter(y=x[3], name = "SP775", marker_color="#DBF227"))

fig.add_trace(go.Bar(y=attr_ig.squeeze().sum(axis=0), name = "Attribution",marker_color=np.where(attr_ig.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False),secondary_y=True)

fig.update_yaxes(title= "Intensity")
fig.update_yaxes(title_text="Attribution", secondary_y=True)
fig.update_xaxes(title= "Timestep")

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y=attr_ig.squeeze()[0],marker_color=np.where(attr_ig.squeeze()[0]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.update_yaxes(title= None, row=2, col=1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=1)
fig.update_xaxes(title= None, row=2, col=1)

fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_ig.squeeze()[1],marker_color=np.where(attr_ig.squeeze()[1]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.update_yaxes(title= None, row=2, col=2)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=2)
fig.update_xaxes(title= None, row=2, col=2)

fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_ig.squeeze()[2],marker_color=np.where(attr_ig.squeeze()[2]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None, row=2, col=3)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=3)
fig.update_xaxes(title= None, row=2, col=3)

fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=4)
fig.add_trace(go.Bar(y=attr_ig.squeeze()[3],marker_color=np.where(attr_ig.squeeze()[3]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=4)

fig.update_yaxes(title= None, row=2, col=4)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=4)
fig.update_xaxes(title= None, row=2, col=4)

fig.update_layout(
    title= format_title("Perovskite 1D Model (Backflow patches excluded)", 
    "Predicted PCE: " + str(y) + " / Method: Integrated Gradients / Infidelity = " 
    + str(*np.round(infid_ig.numpy(),4)) + " ("+ u"\u03C3" + "(" + u"\u03B5" + ") = " + str(std_noise) + ")" + " / Sensitivity = " + str(*np.round(sens_ig.numpy(),4))),
    legend_title = "Wavelength",
    title_y = 0.965,
    title_x = 0.035,
    template = "plotly_white",
    height=600, width=2200
)

fig.write_image("xai/images/1D/1D_nb_ig.png", scale=2)

fig.show()


In [28]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=4, 
                    specs=[[{"secondary_y": True, "colspan": 4}, None,None,None],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Total Attribution"), format_title("", "Attribution ND"), format_title("", "Attribution LP725"), format_title("", "Attribution LP780"), format_title("", "Attribution SP775"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND", marker_color="#042940"))
fig.add_trace(go.Scatter(y=x[1], name = "LP725", marker_color="#005C53"))
fig.add_trace(go.Scatter(y=x[2], name = "LP780", marker_color="#9FC131"))
fig.add_trace(go.Scatter(y=x[3], name = "SP775", marker_color="#DBF227"))

fig.add_trace(go.Bar(y=attr_lime.squeeze().sum(axis=0), name = "Attribution",marker_color=np.where(attr_lime.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False),secondary_y=True)

fig.update_yaxes(title= "Intensity")
fig.update_yaxes(title_text="Attribution", secondary_y=True)
fig.update_xaxes(title= "Timestep")

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y=attr_lime.squeeze()[0],marker_color=np.where(attr_lime.squeeze()[0]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.update_yaxes(title= None, row=2, col=1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=1)
fig.update_xaxes(title= None, row=2, col=1)

fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_lime.squeeze()[1],marker_color=np.where(attr_lime.squeeze()[1]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.update_yaxes(title= None, row=2, col=2)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=2)
fig.update_xaxes(title= None, row=2, col=2)

fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_lime.squeeze()[2],marker_color=np.where(attr_lime.squeeze()[2]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None, row=2, col=3)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=3)
fig.update_xaxes(title= None, row=2, col=3)

fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=4)
fig.add_trace(go.Bar(y=attr_lime.squeeze()[3],marker_color=np.where(attr_lime.squeeze()[3]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=4)

fig.update_yaxes(title= None, row=2, col=4)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=4)
fig.update_xaxes(title= None, row=2, col=4)

fig.update_layout(
    title= format_title("Perovskite 1D Model (Backflow patches excluded)", 
    "Predicted PCE: " + str(y) + " / Method: Lime (" + u"\u237A" + " = " + str(a) + ", exp. kernel width = " + str(kernel_width) + 
    ") / Infidelity = " + str(*np.round(infid_lime.numpy(),4)) + " ("+ u"\u03C3" + "(" + u"\u03B5" + ") = " + str(std_noise) + ")" + " / Sensitivity = " + str(*np.round(sens_lime.numpy(),4))),
    legend_title = "Wavelength",
    title_y = 0.965,
    title_x = 0.035,
    template = "plotly_white",
    height=600, width=2200
)

fig.write_image("xai/images/1D/1D_nb_lime.png", scale=2)

fig.show()


In [30]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=4, 
                    specs=[[{"secondary_y": True, "colspan": 4}, None,None,None],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Total Attribution"), format_title("", "Attribution ND"), format_title("", "Attribution LP725"), format_title("", "Attribution LP780"), format_title("", "Attribution SP775"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND", marker_color="#042940"))
fig.add_trace(go.Scatter(y=x[1], name = "LP725", marker_color="#005C53"))
fig.add_trace(go.Scatter(y=x[2], name = "LP780", marker_color="#9FC131"))
fig.add_trace(go.Scatter(y=x[3], name = "SP775", marker_color="#DBF227"))

fig.add_trace(go.Bar(y=attr_ks.squeeze().sum(axis=0), name = "Attribution",marker_color=np.where(attr_ks.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False),secondary_y=True)

fig.update_yaxes(title= "Intensity")
fig.update_yaxes(title_text="Attribution", secondary_y=True)
fig.update_xaxes(title= "Timestep")

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y=attr_ks.squeeze()[0],marker_color=np.where(attr_ks.squeeze()[0]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.update_yaxes(title= None, row=2, col=1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=1)
fig.update_xaxes(title= None, row=2, col=1)

fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_ks.squeeze()[1],marker_color=np.where(attr_ks.squeeze()[1]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.update_yaxes(title= None, row=2, col=2)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=2)
fig.update_xaxes(title= None, row=2, col=2)

fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_ks.squeeze()[2],marker_color=np.where(attr_ks.squeeze()[2]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None, row=2, col=3)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=3)
fig.update_xaxes(title= None, row=2, col=3)

fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=4)
fig.add_trace(go.Bar(y=attr_ks.squeeze()[3],marker_color=np.where(attr_ks.squeeze()[3]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=4)

fig.update_yaxes(title= None, row=2, col=4)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=4)
fig.update_xaxes(title= None, row=2, col=4)

fig.update_layout(
    title= format_title("Perovskite 1D Model (Backflow patches excluded)", 
    "Predicted PCE: " + str(y) + " / Method: Kernel SHAP with Noise Tunnel (" + u"\u03C3" + " = " + str(stdev) +") / Infidelity = " + str(*np.round(infid_ks.numpy(),4)) + " ("+ u"\u03C3" + "(" + u"\u03B5" + ") = " 
    + str(std_noise) + ")" + " / Sensitivity = " + str(*np.round(sens_ks.numpy(),4)) ),
    legend_title = "Wavelength",
    title_y = 0.965,
    title_x = 0.035,
    template = "plotly_white",
    height=600, width=2200
)

fig.write_image("xai/images/1D/1D_nb_ks.png", scale=2)

fig.show()


In [23]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=4, 
                    specs=[[{"secondary_y": True, "colspan": 4}, None,None,None],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Total Attribution"), format_title("", "Attribution ND"), format_title("", "Attribution LP725"), format_title("", "Attribution LP780"), format_title("", "Attribution SP775"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND", marker_color="#042940"))
fig.add_trace(go.Scatter(y=x[1], name = "LP725", marker_color="#005C53"))
fig.add_trace(go.Scatter(y=x[2], name = "LP780", marker_color="#9FC131"))
fig.add_trace(go.Scatter(y=x[3], name = "SP775", marker_color="#DBF227"))

fig.add_trace(go.Bar(y=attr_gbp.squeeze().sum(axis=0), name = "Attribution",marker_color=np.where(attr_gbp.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False),secondary_y=True)

fig.update_yaxes(title= "Intensity")
fig.update_yaxes(title_text="Attribution", secondary_y=True)
fig.update_xaxes(title= "Timestep")

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y= attr_gbp.squeeze()[0],marker_color=np.where(attr_gbp.squeeze()[0]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.update_yaxes(title= None, row=2, col=1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=1)
fig.update_xaxes(title= None, row=2, col=1)

fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_gbp.squeeze()[1],marker_color=np.where(attr_gbp.squeeze()[1]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.update_yaxes(title= None, row=2, col=2)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=2)
fig.update_xaxes(title= None, row=2, col=2)

fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_gbp.squeeze()[2],marker_color=np.where(attr_gbp.squeeze()[2]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None, row=2, col=3)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=3)
fig.update_xaxes(title= None, row=2, col=3)

fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=4)
fig.add_trace(go.Bar(y=attr_gbp.squeeze()[3],marker_color=np.where(attr_gbp.squeeze()[3]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=4)

fig.update_yaxes(title= None, row=2, col=4)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=4)
fig.update_xaxes(title= None, row=2, col=4)

fig.update_layout(
    title= format_title("Perovskite 1D Model (Backflow patches excluded)", 
    "Predicted PCE: " + str(y) + " / Method: Guided Backprob / Infidelity = " + str(*np.round(infid_gbp.numpy(),4)) + " ("+ u"\u03C3" + "(" + u"\u03B5" + ") = " + str(std_noise) + ")" 
    + " / Sensitivity = " + str(*np.round(sens_gbp.numpy(),4))),
    legend_title = "Wavelength",
    title_y = 0.965,
    title_x = 0.035,
    template = "plotly_white",
    height=600, width=2200
)

fig.write_image("xai/images/1D/1D_nb_gbp.png", scale=2)

fig.show()


In [24]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=4, 
                    specs=[[{"secondary_y": True, "colspan": 4}, None,None,None],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Total Attribution"), format_title("", "Attribution ND"), format_title("", "Attribution LP725"), format_title("", "Attribution LP780"), format_title("", "Attribution SP775"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND", marker_color="#042940"))
fig.add_trace(go.Scatter(y=x[1], name = "LP725", marker_color="#005C53"))
fig.add_trace(go.Scatter(y=x[2], name = "LP780", marker_color="#9FC131"))
fig.add_trace(go.Scatter(y=x[3], name = "SP775", marker_color="#DBF227"))

fig.add_trace(go.Bar(y=attr_ggc.squeeze().sum(axis=0), name = "Attribution",marker_color=np.where(attr_ggc.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False),secondary_y=True)

fig.update_yaxes(title= "Intensity")
fig.update_yaxes(title_text="Attribution", secondary_y=True)
fig.update_xaxes(title= "Timestep")

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y= attr_ggc.squeeze()[0],marker_color=np.where(attr_ggc.squeeze()[0]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.update_yaxes(title= None, row=2, col=1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=1)
fig.update_xaxes(title= None, row=2, col=1)

fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_ggc.squeeze()[1],marker_color=np.where(attr_ggc.squeeze()[1]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.update_yaxes(title= None, row=2, col=2)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=2)
fig.update_xaxes(title= None, row=2, col=2)

fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_ggc.squeeze()[2],marker_color=np.where(attr_ggc.squeeze()[2]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None, row=2, col=3)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=3)
fig.update_xaxes(title= None, row=2, col=3)

fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.5,showlegend=False), row=2, col=4)
fig.add_trace(go.Bar(y=attr_ggc.squeeze()[3],marker_color=np.where(attr_ggc.squeeze()[3]<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=4)

fig.update_yaxes(title= None, row=2, col=4)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col=4)
fig.update_xaxes(title= None, row=2, col=4)

fig.update_layout(
    title= format_title("Perovskite 1D Model (Backflow patches excluded)", 
    "Predicted PCE: " + str(y) + " / Method: Guided GradCAM / Infidelity = " + str(*np.round(infid_ggc.numpy(),4)) + " ("+ u"\u03C3" + "(" + u"\u03B5" + ") = " + str(std_noise) + ")" + 
    " / Sensitivity = " + str(*np.round(sens_ggc.numpy(),4))),
    legend_title = "Wavelength",
    title_y = 0.965,
    title_x = 0.035,
    template = "plotly_white",
    height=600, width=2200
)

fig.write_image("xai/images/1D/1D_nb_ggc.png", scale=2)

fig.show()


In [31]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=3, 
                    specs=[[{"secondary_y": True}, {"secondary_y": True},{"secondary_y": True}],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Expected Gradients (" + str(*np.round(infid_eg.numpy(),4)) + ", " + str(*np.round(sens_eg.numpy(),4)) + ")"), 
                    format_title("", "Integrated Gradients (" + str(*np.round(infid_ig.numpy(),4)) + ", " + str(*np.round(sens_ig.numpy(),4)) + ")"), 
                    format_title("", "LIME (" + str(*np.round(infid_lime.numpy(),4)) + ", " + str(*np.round(sens_lime.numpy(),4)) + ")"), 
                    format_title("", "Kernel SHAP w/ Noise Tunnel (" + str(*np.round(infid_ks.numpy(),4)) + ", " + str(*np.round(sens_ks.numpy(),4)) + ")"), 
                    format_title("", "Guided Backprop (" + str(*np.round(infid_gbp.numpy(),4)) + ", " + str(*np.round(sens_gbp.numpy(),4)) + ")"), 
                    format_title("", "Guided GradCAM (" + str(*np.round(infid_ggc.numpy(),4)) + ", " + str(*np.round(sens_ggc.numpy(),4)) + ")"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Bar(y=attr_eg.squeeze().sum(axis=0),marker_color=np.where(attr_eg.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=1, col=1)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Bar(y=attr_ig.squeeze().sum(axis=0),marker_color=np.where(attr_ig.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=1, col=2)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Bar(y=attr_lime.squeeze().sum(axis=0),marker_color=np.where(attr_lime.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=1, col=3)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y=attr_ks.squeeze().sum(axis=0),marker_color=np.where(attr_ks.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_gbp.squeeze().sum(axis=0),marker_color=np.where(attr_gbp.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_ggc.squeeze().sum(axis=0),marker_color=np.where(attr_ggc.squeeze().sum(axis=0)<0, 'red', 'green'), opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None)
fig.update_yaxes(title_text=None, secondary_y=True)
fig.update_xaxes(title= None)

fig.update_yaxes(title= "Intensity", row=1, col =1)
fig.update_yaxes(title_text=None, secondary_y=True, row=1, col =1)
fig.update_yaxes(title_text="Attribution", secondary_y=True, row=1, col =3)
fig.update_yaxes(title= "Intensity", row=2, col =1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col =1)
fig.update_yaxes(title_text="Attribution", secondary_y=True, row=2, col =3)
fig.update_xaxes(title= "Timesteps",row=2,col=2)

fig.update_layout(
    title= format_title("Method Comparision: Summed Total Attribution", "Perovskite 1D Model (Backflow patches excluded) / Predicted PCE: " + str(y) + " / (Infidelity, Sensitivity)"),
    legend_title = None,
    title_y = 0.96,
    template = "plotly_white",
    height=800, width=2000
)

fig.write_image("xai/images/1D/1D_nb_cmp_sum.png", scale=2)

fig.show()


In [32]:
import plotly.graph_objects as go

def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=3, 
                    specs=[[{"secondary_y": True}, {"secondary_y": True},{"secondary_y": True}],
                           [{"secondary_y": True},{"secondary_y": True},{"secondary_y": True}]],
                    subplot_titles=(format_title("", "Expected Gradients (" + str(*np.round(infid_eg.numpy(),4)) + ", " + str(*np.round(sens_eg.numpy(),4)) + ")"), 
                    format_title("", "Integrated Gradients (" + str(*np.round(infid_ig.numpy(),4)) + ", " + str(*np.round(sens_ig.numpy(),4)) + ")"), 
                    format_title("", "LIME (" + str(*np.round(infid_lime.numpy(),4)) + ", " + str(*np.round(sens_lime.numpy(),4)) + ")"), 
                    format_title("", "Kernel SHAP w/ Noise Tunnel (" + str(*np.round(infid_ks.numpy(),4)) + ", " + str(*np.round(sens_ks.numpy(),4)) + ")"), 
                    format_title("", "Guided Backprop (" + str(*np.round(infid_gbp.numpy(),4)) + ", " + str(*np.round(sens_gbp.numpy(),4)) + ")"), 
                    format_title("", "Guided GradCAM (" + str(*np.round(infid_ggc.numpy(),4)) + ", " + str(*np.round(sens_ggc.numpy(),4)) + ")"))
                    )

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=1)
fig.add_trace(go.Bar(y=attr_eg.squeeze().abs().sum(axis=0),marker_color= "#042940", opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=1, col=1)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=2)
fig.add_trace(go.Bar(y=attr_ig.squeeze().abs().sum(axis=0),marker_color= "#005C53", opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=1, col=2)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=1, col=3)
fig.add_trace(go.Bar(y=attr_lime.squeeze().abs().sum(axis=0),marker_color= "#D6D58E", opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=1, col=3)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(y=attr_ks.squeeze().abs().sum(axis=0),marker_color= "#D6D58E", opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=1)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=2)
fig.add_trace(go.Bar(y=attr_gbp.squeeze().abs().sum(axis=0),marker_color= "#9FC131", opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=2)

fig.add_trace(go.Scatter(y=x[0], name = "ND",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Scatter(y=x[1], name = "LP725",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Scatter(y=x[2], name = "LP780",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Scatter(y=x[3], name = "SP775",marker_color="grey", opacity = 0.3,showlegend=False), row=2, col=3)
fig.add_trace(go.Bar(y=attr_ggc.squeeze().abs().sum(axis=0),marker_color= "#DBF227", opacity = 0.5,showlegend=False,marker_line_width=0),secondary_y=True, row=2, col=3)

fig.update_yaxes(title= None)
fig.update_yaxes(title_text=None, secondary_y=True)
fig.update_xaxes(title= None)

fig.update_yaxes(title= "Intensity", row=1, col =1)
fig.update_yaxes(title_text=None, secondary_y=True, row=1, col =1)
fig.update_yaxes(title_text="Attribution", secondary_y=True, row=1, col =3)
fig.update_yaxes(title= "Intensity", row=2, col =1)
fig.update_yaxes(title_text=None, secondary_y=True, row=2, col =1)
fig.update_yaxes(title_text="Attribution", secondary_y=True, row=2, col =3)
fig.update_xaxes(title= "Timesteps",row=2,col=2)

fig.update_layout(
    title= format_title("Method Comparision: Absolute Total Attribution", "Perovskite 1D Model (Backflow patches excluded) / Predicted PCE: " + str(y) + " / (Infidelity, Sensitivity)"),
    legend_title = None,
    title_y = 0.96,
    title_x = 0.035,
    template = "plotly_white",
    height=800, width=2000
)

fig.write_image("xai/images/1D/1D_nb_cmp_abs.png", scale=2)

fig.show()